# Homework 4
## Mengxiang Jiang

The majority of the code here comes from the [Companion Jupyter notebooks for the book "Deep Learning with Python"](https://github.com/fchollet/deep-learning-with-python-notebooks) by François Chollet, specifically chapter 2.
If the code comes from somewhere else it will have a source linked to the original.\
[Video Link](https://youtu.be/gmHJk2IpYro)

**Computational Graph**

<img src="comp_graph.png" width="1200">

**Using Tensorflow to Calculate Partial Derivatives**

In [1]:
import tensorflow as tf

2023-02-17 18:10:45.132738: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 18:10:45.204676: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
x = tf.Variable(-1.)
y = tf.Variable(2.)
z = tf.Variable(3.)
with tf.GradientTape() as tape:
    f = x*y + z**3 + x
grad_of_f_wrt_x_y_z = tape.gradient(f, [x, y, z])
print(f"partial derivative wrt to x: {grad_of_f_wrt_x_y_z[0]}")
print(f"partial derivative wrt to y: {grad_of_f_wrt_x_y_z[1]}")
print(f"partial derivative wrt to z: {grad_of_f_wrt_x_y_z[2]}")

partial derivative wrt to x: 3.0
partial derivative wrt to y: -1.0
partial derivative wrt to z: 27.0


2023-02-17 18:10:45.963821: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 18:10:46.258760: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7815 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


## Looking back at our first example

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

history = model.fit(train_images, train_labels, epochs=5, batch_size=128)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"test_acc: {test_acc}")

Epoch 1/5
  1/469 [..............................] - ETA: 5:28 - loss: 2.3972 - accuracy: 0.0469

2023-02-17 18:10:47.231656: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-17 18:10:47.232899: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f5bb9f93c00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-17 18:10:47.232910: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2023-02-17 18:10:47.235059: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-17 18:10:47.286987: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


469/469 [==============================] - 1s 922us/step - loss: 0.2665 - accuracy: 0.9232
Epoch 2/5
469/469 [==============================] - 0s 894us/step - loss: 0.1084 - accuracy: 0.9677
Epoch 3/5
469/469 [==============================] - 0s 871us/step - loss: 0.0710 - accuracy: 0.9787
Epoch 4/5
469/469 [==============================] - 0s 899us/step - loss: 0.0515 - accuracy: 0.9844
Epoch 5/5
313/313 [==============================] - 0s 668us/step - loss: 0.0660 - accuracy: 0.9795
test_acc: 0.9794999957084656


### Reimplementing our first example from scratch in TensorFlow
#### A simple Dense class

In [4]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation

        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)

        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property
    def weights(self):
        return [self.W, self.b]

#### A simple Sequential class

In [5]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
           x = layer(x)
        return x

    @property
    def weights(self):
       weights = []
       for layer in self.layers:
           weights += layer.weights
       return weights

In [6]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

#### A batch generator

In [7]:
import math
import numpy as np

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        p = np.random.permutation(len(images))
        self.index = 0
        self.images = images[p]
        self.labels = labels[p]
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

### Running one training step

In [8]:
def one_training_step(model, images_batch, labels_batch, learning_rate, past_velocity, momentum):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    past_velocity = update_weights(gradients, model.weights, learning_rate, past_velocity, momentum)
    return average_loss, past_velocity

In [9]:
def update_weights(gradients, weights, learning_rate, past_velocity, momentum):
    for g, w in zip(gradients, weights):
        velocity = past_velocity * momentum - learning_rate * g
        w.assign(w + momentum * velocity - learning_rate * g)
        past_velocity = velocity
        return past_velocity

### The full training loop

In [10]:
def fit(model, images, labels, epochs, batch_size=128, learning_rate=1e-1, past_velocity=0, momentum=0.1):
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss, past_velocity = one_training_step(model, images_batch, labels_batch, learning_rate, past_velocity, momentum)
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter}: {loss:.2f}")

In [11]:
fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0
loss at batch 0: 5.69
loss at batch 100: 0.81
loss at batch 200: 0.63
loss at batch 300: 0.56
loss at batch 400: 0.48
Epoch 1
loss at batch 0: 0.45
loss at batch 100: 0.39
loss at batch 200: 0.37
loss at batch 300: 0.40
loss at batch 400: 0.38
Epoch 2
loss at batch 0: 0.36
loss at batch 100: 0.53
loss at batch 200: 0.33
loss at batch 300: 0.41
loss at batch 400: 0.34
Epoch 3
loss at batch 0: 0.47
loss at batch 100: 0.47
loss at batch 200: 0.45
loss at batch 300: 0.41
loss at batch 400: 0.32
Epoch 4
loss at batch 0: 0.26
loss at batch 100: 0.39
loss at batch 200: 0.29
loss at batch 300: 0.46
loss at batch 400: 0.45
Epoch 5
loss at batch 0: 0.30
loss at batch 100: 0.40
loss at batch 200: 0.34
loss at batch 300: 0.34
loss at batch 400: 0.36
Epoch 6
loss at batch 0: 0.46
loss at batch 100: 0.30
loss at batch 200: 0.30
loss at batch 300: 0.22
loss at batch 400: 0.25
Epoch 7
loss at batch 0: 0.40
loss at batch 100: 0.24
loss at batch 200: 0.30
loss at batch 300: 0.40
loss at batch 40

### Evaluating the model

In [12]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f}")

accuracy: 0.92
